# This notebook shows an example where a set of electrodes are selected from a dataset and then LFP is extracted from those electrodes and then written to a new NWB file


## The following steps must be completed before running this notebook:
### 1) Set up access to a DataJoint database. See https://tutorials.datajoint.io/
### 2) Install pynwb. 
    See https://pynwb.readthedocs.io/en/stable/getting_started.html#installation
    Note: currently this requires pynwb 1.3.3 or above and hdmf 2.0.1 or above, so use the latest development versions, not the conda versions
### 3) Install the ndx-fl-novela package. 
     conda install -c novelakrk ndx-fl-novela
### 4) Download the example franklab NWB file from DropBox. 
    https://www.dropbox.com/to_be_updated

#### Load all of the relevant modules and set the environment variables. 
Note that the datadir and datadir/analysis must exist

In [11]:
%env DJ_SUPPORT_FILEPATH_MANAGEMENT=TRUE
%load_ext autoreload
%autoreload 2


import pynwb
import os

#DataJoint and DataJoint schema
import datajoint as dj

# the commands below can be run once to update your global configuration

#dj.config["enable_python_native_blobs"] = True
#dj.config["database.user"] = 'root'
#dj.config["database.password"] = 'tutorial'
#dj.config.save_global()

import nwb_datajoint as nd
import ndx_franklab_novela.probe

data_dir = '/Users/loren/data/nwb_builder_test_data'
os.environ['SPYGLASS_BASE_DIR'] = data_dir
os.environ['KACHERY_STORAGE_DIR'] = os.path.join(data_dir, 'kachery-storage')



env: DJ_SUPPORT_FILEPATH_MANAGEMENT=TRUE
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Next we select the NWB file, which corresponds to the dataset we want to extract LFP from

In [12]:
nwb_file_names = nd.common.Nwbfile().fetch('nwb_file_name')
# take the first one for this demonstration
nwb_file_name = nwb_file_names[0]
print(nwb_file_name)

beans20190718.nwb


#### Create the standard LFP Filters. This only needs to be done once.

In [3]:
nd.common.FirFilter().create_standard_filters()

### Select every 16th electrode for LFP

In [4]:
electrode_ids = nd.common.Electrode.fetch('electrode_id')
lfp_electrode_ids = electrode_ids[range(0, len(electrode_ids), 16)]
nd.common.LFPSelection().set_lfp_electrodes(nwb_file_name, lfp_electrode_ids.tolist())


About to delete:
Nothing to delete


In [5]:
nd.common.LFPSelection().LFPElectrode()

nwb_file_name the name of the NWB file,electrode_group_name electrode group name from NWBFile,electrode_id the unique number for this electrode
beans20190718.nwb,0,0
beans20190718.nwb,0,16
beans20190718.nwb,0,32
beans20190718.nwb,0,48
beans20190718.nwb,0,64
beans20190718.nwb,0,80
beans20190718.nwb,0,96
beans20190718.nwb,0,112
beans20190718.nwb,1,128
beans20190718.nwb,1,144


### Or select one electrode for LFP


In [6]:
nd.common.LFPSelection().set_lfp_electrodes(nwb_file_name, [2])

About to delete:
Nothing to delete


### Populate the LFP table. Note that this takes 2 hours or so on a laptop if you use all electrodes

In [3]:
nd.common.LFP().populate()

writing new NWB file beans20190718_00000001.nwb
Output array should have shape (1091950, 16) and dtype <f8
Checking output array shape is disabled, make sure portion of output array has shape (1091950, 16)


### Now that we've created the LFP object we can perform a second level of filtering for a band of interest, in this case the theta band
We first need to create the filter

In [10]:
lfp_sampling_rate = (nd.common.LFP() & {'nwb_file_name' : nwb_file_name}).fetch1('lfp_sampling_rate')
filter_name = 'Theta 5-11 Hz'
nd.common.FirFilter().add_filter(filter_name, lfp_sampling_rate, 'bandpass', [4, 5, 11, 12], 'theta filter for 1 Khz data')

In [11]:
nd.common.FirFilter()

filter_name descriptive name of this filter,filter_sampling_rate sampling rate for this filter,filter_type,filter_low_stop lowest frequency for stop band for low frequency side of filter,filter_low_pass lowest frequency for pass band of low frequency side of filter,filter_high_pass highest frequency for pass band for high frequency side of filter,filter_high_stop highest frequency for stop band of high frequency side of filter,filter_comments comments about the filter,filter_band_edges numpy array containing the filter bands (redundant with individual parameters),filter_coeff numpy array containing the filter coefficients
LFP 0-400 Hz,20000,lowpass,0.0,0.0,400.0,425.0,standard LFP filter for 20 KHz data,=BLOB=,=BLOB=
LFP 0-400 Hz,30000,lowpass,0.0,0.0,400.0,425.0,standard LFP filter for 20 KHz data,=BLOB=,=BLOB=
Theta 5-11 Hz,1000,lowpass,4.0,5.0,11.0,12.0,theta filter for 1 Khz data,=BLOB=,=BLOB=


Next we add an entry for the LFP Band and the electrodes we want to filter

In [12]:
# assume that we've filtered these electrodes; change this if not
lfp_band_electrode_ids = [0, 16, 32]

# set the interval list name corresponding to the second epoch (a run session)
interval_list_name = '02_r1'

# set the reference to -1 to indicate no reference for all channels
ref_elect = [-1]

# desired sampling rate
lfp_band_sampling_rate = lfp_sampling_rate // 10

In [14]:
nd.common.LFPBandSelection().set_lfp_band_electrodes(nwb_file_name, lfp_band_electrode_ids, filter_name, interval_list_name, ref_elect, lfp_band_sampling_rate)

Check to make sure it worked

In [15]:
nd.common.LFPBandSelection().LFPBandElectrode()

nwb_file_name the name of the NWB file,filter_name descriptive name of this filter,filter_sampling_rate sampling rate for this filter,electrode_group_name electrode group name from NWBFile,electrode_id the unique number for this electrode,reference_elect_id the reference electrode to use; -1 for no reference
beans20190718.nwb,Theta 5-11 Hz,1000,0,0,-1
beans20190718.nwb,Theta 5-11 Hz,1000,0,16,-1
beans20190718.nwb,Theta 5-11 Hz,1000,0,32,-1


In [ ]:
(nd.common.LFP() & {'nwb_file_name' : nwb_file_name}).fetch_nwb(limit=1)[0]['lfp']

In [4]:
nd.common.LFPBand().populate()

writing new NWB file beans20190718_00000002.nwb
[[1.56349543e+09 1.56349711e+09]]
1563495426.1787295 1563497107.1339033


ValueError: 'input_index_bounds' [1091950, 1091950] is not strictly increasing

### Now we can plot the original signal, the LFP filtered trace, and the theta filtered trace together.
Much of the code below could be replaced by a function calls that would return the data from each electrical series

In [5]:
import matplotlib.pyplot as plt
import numpy as np

/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
#get the three electrical series objects and the indeces of the electrodes we band pass filtered
orig_eseries = (nd.common.Raw() & {'nwb_file_name' : nwb_file_name}).fetch_nwb()[0]['raw']
orig_elect_indeces = nd.common.get_electrode_indeces(orig_eseries, lfp_band_electrode_ids)

lfp_eseries = (nd.common.LFP() & {'nwb_file_name' : nwb_file_name}).fetch_nwb()[0]['lfp']
lfp_elect_indeces = nd.common.get_electrode_indeces(lfp_eseries, lfp_band_electrode_ids)

lfp_band_eseries = (nd.common.LFPBand() & {'nwb_file_name' : nwb_file_name}).fetch_nwb()[0]['filtered_data']
lfp_band_elect_indeces = nd.common.get_electrode_indeces(lfp_band_eseries, lfp_band_electrode_ids)


/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'lfp_band_electrode_ids' is not defined

In [8]:
(nd.common.Raw() & {'nwb_file_name' : nwb_file_name}).fetch_nwb()

/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[{'nwb_file_name': 'beans20190718.nwb',
  'interval_list_name': 'raw data valid times',
  'raw_object_id': '4d3b3798-27c7-4c6d-a9bc-ff22161f0a86',
  'sampling_rate': 20000.0,
  'comments': 'sample comment',
  'description': 'Electrical series registered on electrode',
  'raw': e-series pynwb.ecephys.ElectricalSeries at 0x140257258944976
  Fields:
    comments: sample comment
    conversion: 1.0
    data: <HDF5 dataset "data": shape (138007758, 256), type "<i2">
    description: Electrical series registered on electrode
    electrodes: electrodes <class 'hdmf.common.table.DynamicTableRegion'>
    interval: 1
    resolution: -1.0
    timestamps: <HDF5 dataset "timestamps": shape (138007758,), type "<f8">
    timestamps_unit: seconds
    unit: volts}]

In [10]:
# get a list of times for the first run epoch and then select a 2 second interval 100 seconds from the beginning
run1times = (nd.common.IntervalList & {'interval_list_name' : '01_s1'}).fetch1('valid_times')
plottimes = [run1times[0][0] + 101, run1times[0][0] + 102]

/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# get the time indeces for each dataset
orig_time_ind = np.argwhere(np.logical_and(orig_eseries.timestamps > plottimes[0], orig_eseries.timestamps < plottimes[1]))
lfp_time_ind = np.argwhere(np.logical_and(lfp_eseries.timestamps > plottimes[0], lfp_eseries.timestamps < plottimes[1]))
lfp_band_time_ind = np.argwhere(np.logical_and(lfp_band_eseries.timestamps > plottimes[0], lfp_band_eseries.timestamps < plottimes[1]))

In [ ]:
plt.plot(orig_eseries.timestamps[orig_time_ind], orig_eseries.data[orig_time_ind,orig_elect_indeces[1]], 'k-')
plt.plot(lfp_eseries.timestamps[lfp_time_ind], lfp_eseries.data[lfp_time_ind,lfp_elect_indeces[1]], 'b-')
plt.plot(lfp_band_eseries.timestamps[lfp_band_time_ind], lfp_band_eseries.data[lfp_band_time_ind,lfp_band_elect_indeces[1]], 'r-')
plt.xlabel('Time (sec)')
plt.ylabel('Amplitude (AD units)')

plt.show()

In [7]:
nd.common.Session().drop()

NetworkXError: The node `common_session`.`_session` is not in the graph.

In [6]:
common_lab.Session().drop()

[autoreload of nwb_datajoint.common.common_nwbfile failed: Traceback (most recent call last):
  File "/Users/loren/opt/anaconda3/envs/nwb_datajoint/lib/python3.7/site-packages/datajoint/declare.py", line 142, in compile_foreign_key
    ref = eval(result.ref_table, context)
  File "<string>", line 1
    analysis_file_sha1: varchar(40) # the sha1 hash of the file
                      ^
SyntaxError: invalid syntax

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/loren/opt/anaconda3/envs/nwb_datajoint/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/loren/opt/anaconda3/envs/nwb_datajoint/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/Users/loren/opt/anaconda3/envs/nwb_datajoint/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(mod

NameError: name 'common_lab' is not defined